In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from keras.applications.inception_v3 import preprocess_input
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,load_img,img_to_array



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

dataset_path='/content/gdrive/MyDrive/Project/Axial/'

Mounted at /content/gdrive


In [ ]:
datagen = ImageDataGenerator(validation_split=0.2,rescale=1./255)

In [ ]:
train_batches = datagen.flow_from_directory(dataset_path,
                                            target_size=(256,170),
                                            batch_size=10,class_mode='categorical',subset='training')
test_batches = datagen.flow_from_directory(dataset_path,
                                      target_size=(256,170),
                                      batch_size=10,class_mode='categorical',subset='validation')

Found 4156 images belonging to 3 classes.
Found 1038 images belonging to 3 classes.


In [ ]:
def plotImages(images_arr):
  fig, axes =plt.subplots(1,5)
  axes=axes.flatten()
  for img,ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show

In [ ]:
imgs, lables= train_batches[0]

In [ ]:
class_name={"Alzhiemer's Diesease","Cognitive Impairment","Cognitive Normal"}

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Flatten,Conv2D,MaxPool2D,Softmax,Dropout

In [ ]:
model = Sequential()
model.add(Conv2D(64,(4,4), input_shape=imgs.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(3,3)))

model.add(Conv2D(32,(4,4)))
model.add(Activation("relu"))
model.add(Dropout(0.1))

model.add(Conv2D(64,(4,4)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(3,3)))


model.add(Flatten())

model.add(Dense(64))

model.add(Dense(64))
model.add(Dense(3))

model.add(Softmax())

model.compile(loss="binary_crossentropy",
               optimizer="adam",
              metrics=['accuracy'])



In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor="accuracy",
                   min_delta=0.01 ,
                   patience = 2,
                   verbose=1)

In [ ]:
model.fit_generator(train_batches,steps_per_epoch=100,epochs=5,
                    callbacks=es)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
100/100 [==============================] - 123s 1s/step - loss: 0.6295 - accuracy: 0.4770
Epoch 2/5
100/100 [==============================] - 106s 1s/step - loss: 0.5689 - accuracy: 0.5392
Epoch 3/5
100/100 [==============================] - 111s 1s/step - loss: 0.4347 - accuracy: 0.6860
Epoch 4/5
100/100 [==============================] - 104s 1s/step - loss: 0.3009 - accuracy: 0.8060
Epoch 5/5
100/100 [==============================] - 105s 1s/step - loss: 0.2479 - accuracy: 0.8460


In [ ]:
model.fit_generator(train_batches,steps_per_epoch=100,epochs=5,
                    callbacks=es)
train_batches.class_indices

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


100/100 [==============================] - 104s 1s/step - loss: 0.1350 - accuracy: 0.9150
Epoch 2/5
100/100 [==============================] - 104s 1s/step - loss: 0.1048 - accuracy: 0.9370
Epoch 3/5
100/100 [==============================] - 104s 1s/step - loss: 0.0933 - accuracy: 0.9528
Epoch 4/5
100/100 [==============================] - 105s 1s/step - loss: 0.0871 - accuracy: 0.9470
Epoch 5/5
100/100 [==============================] - 105s 1s/step - loss: 0.0654 - accuracy: 0.9640


{'AD': 0, 'CI': 1, 'CN': 2}

In [ ]:
path = '/content/gdrive/My Drive/Project/Axial/CI/CI002_S_0729a092.png'
img  = load_img(path,target_size=(256,170))

i = img_to_array(img)

i = preprocess_input(i)

input_arr = np.array([i])
input_arr.shape

pred = np.argmax(model.predict(input_arr))
print(pred)
plt.imshow(input_arr[0])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


0


In [ ]:
print(model.evaluate(test_batches))
print(model.evaluate(train_batches))

104/104 [==============================] - 24s 232ms/step - loss: 1.2198 - accuracy: 0.5039
[1.2198162078857422, 0.5038535594940186]
416/416 [==============================] - 105s 251ms/step - loss: 0.0413 - accuracy: 0.9832
[0.04134439677000046, 0.9831568598747253]
